In [10]:
# Download ANTs-CT analyses for T1 Normative Dataset

import flywheel
fw = flywheel.Client()
collection_id = '5eb5081448fe1b1e5792a7a9'
gear = fw.lookup('gears/antsct-fw')

In [5]:
# Here we get all the sessions within the collection.
# We use `get_session` to return the associated analyses
sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]
print("done")

done


In [74]:
input_t1s = []
no_t1_sessions = []
analysis_ids = []
#sessions_short = [fw.get('5e4fe6e36dea314dcf2a7473'), fw.get('5e5920e5782dc842a9f1e9ee')]

for session in sessions:
    ## Get acquisition
    found = False
    potential_t1s = [None] * len(session.acquisitions()
    for acq in session.acquisitions():
        has_t1 = False
        has_struct_intent = False
        has_mprage = False
        if acq.files and len(acq.files) > 0:
            for i in range(0, len(acq.files)):
                try: 
                    has_t1 = 'T1' in acq.files[i]['classification']['Measurement'] 
                    has_struct_intent = 'Structural' in acq.files[i]['classification']['Intent']
                    has_mprage = 'MPRAGE' in acq.files[i]['classification']['Features']
                except TypeError as te:
                    continue
                except KeyError as ke:
                    continue                
                break                    

        # follow a hierarchy of surest identifiers of the desired T1 acquisition (usually mprage, never setter, etc.)
        if has_mprage:
            potential_t1s.insert(0, acq)
            found = True
            break
        elif "mprage" in acq.label.lower() or 't1w_mpr' in acq.label.lower():
            potential_t1s.insert(0, acq)
            found = True
            break
        elif has_t1 and has_struct_intent and "setter" not in acq.label:
            potential_t1s.append(acq)
            found = True
            break
        else:
            continue  
            
    if not found:
        sub = fw.get(session.parents['subject'])
        no_t1_sessions.append('{}: {}'.format(sub.label, session.label))
    else:
        # add the highest priority acquisition to the input T1s list
        input_t1_acq = potential_t1s[0]
        input_t1s.append(input_t1_acq)

        # Run the gear
        inputs = {'t1_anatomy': input_t1_acq}
        config = {'denoise': True, 'num-threads': 0, 'trim-neck': True}
        #analysis_id = gear.run(analysis_label='antsct_2020-11-10_WT', config=config, inputs=inputs, destination=session)
        #analysis_ids.append(analysis_id)

KeyboardInterrupt: 

In [ ]:
print(len(input_t1s))

In [69]:
for t1 in input_t1s:
    print(t1.label)

T1w_MPR 
T1w_MPR 
T1w_MPR 
T1w_MPR 
T1w_MPR 
T1w_MPR 
T1w_MPR 
T1w_MPR 
T1w_MPR 
T1w_MPR 
T1w_MPR 
T1w_MPR 
T1w_MPR_axial 
T1w_MPR 
T1w_MPR 
T1w_MPR 
T1w_MPR_axial 
T1w_MPR 
T1w_MPR_axial 
T1w_MPR 
T1w_axial 
T1w_Axial 
T1w_MPR 
T1w_MPR_axial 
T1w_MPR 
T1w_Axial 
T1w_MPR
T1w_axial
T1w_MPR_axial
T1w_axial
T1w_Axial
T1w_axial
T1w_axial
T1w_MPR
T1w_MPR
T1w_MPR
T1w_MPR
T1w_MPR
T1w_axial
T1w_MPR
T1w_MPR
T1w_MPR
T1_3D_0.8x0.8x0.8_ND
T1_3D_0.8x0.8x0.8_ND
T1_3D_0.8x0.8x0.8_ND
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8
T1w_MPR
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8_ND
T1w_MPR
T1_3D_0.8x0.8x0.8_ND
T1_3D_0.8x0.8x0.8
T1w_axial
T1w_MPR
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8_ND
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8_ND
T1w_MPR
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8_ND
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8_ND
T1w_MPR
T1_3D_0.8x0.8x0.8_ND
T1_3D_0.8x0.8x0.8_ND
T1_3D_0.8x0.8x0.8
T1_3D_0.8x0.8x0.8
T1w_MPR
T1_3D_0.8x0.8

In [70]:
print(len(no_t1_sessions))
no_t1_sessions

0


[]

In [72]:
for t1 in input_t1s:
    sub = fw.get(t1.parents['subject'])
    ses = fw.get(t1.parents['session'])
    print('{},{},{}'.format(t1.label, sub.label, ses.label))

T1w_MPR ,TOME_3001,Session 1
T1w_MPR ,TOME_3002,Session 1
T1w_MPR ,TOME_3003,Session 1
T1w_MPR ,TOME_3007,Session 1
T1w_MPR ,TOME_3004,Session 1a
T1w_MPR ,TOME_3008,Session 1
T1w_MPR ,TOME_3009,Session 1
T1w_MPR ,TOME_3005,Session 1
T1w_MPR ,TOME_3018,Session 1b
T1w_MPR ,TOME_3012,Session 1a
T1w_MPR ,TOME_3015,Session 1
T1w_MPR ,TOME_3013,Session 1
T1w_MPR_axial ,TOME_3017,Session 1
T1w_MPR ,TOME_3011,Session 1
T1w_MPR ,TOME_3014,Session 1
T1w_MPR ,TOME_3016,Session 1
T1w_MPR_axial ,TOME_3024,Session 1
T1w_MPR ,TOME_3026,Session 1
T1w_MPR_axial ,TOME_3021,Session 1a
T1w_MPR ,TOME_3022,Session 1
T1w_axial ,TOME_3023,Session 1
T1w_Axial ,TOME_3028,Session 1
T1w_MPR ,TOME_3019,Session 1b
T1w_MPR_axial ,TOME_3025,Session 1
T1w_MPR ,TOME_3027,Session 1
T1w_Axial ,TOME_3020,Session 1
T1w_MPR,TOME_3029,Session 1
T1w_axial,TOME_3030,T1 and T2 images
T1w_MPR_axial,TOME_3031,Session 1
T1w_axial,TOME_3032,Session 1
T1w_Axial,TOME_3033,Session 1
T1w_axial,TOME_3035,Session 1
T1w_axial,TOME_3034,Se